In [1]:
import cv2
import numpy as numpy

from sklearn.metrics import pairwise

In [2]:
# Global variable

background = None
accumulated_weigh = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calc_accum_avg(frame, accumulayed_weigh):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return None

    cv2.accumulatedWeighted(frame, background, accumulated_weigh)

In [ ]:
def segment(frame, threshold=25):
    diff = cv2.absdiff(background.astype('uint8'),frame)
    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 1500:
            continue
        else:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
    return frame